In [1]:
import pandas as pd

In [2]:
def split_values(text: str):

    values = [v.split('-') for v in text.split(', ')]

    for item in values:
        if len(item) == 1:
            item.append(1)
    return values

In [3]:
data = pd.read_excel('PQ-297.xlsx', usecols = 'A:B', header = 0, nrows = 5)
data.head()

,Cage,Animals & Count
0,Cage10,"Tiger, Lion-2, Elephant-4, Leopard, Panther"
1,Cage2,"Cheetah, Wolf-8"
2,Cage3,Jaguar-2
3,Cage12,"Monkey-3, Giraffe-6, Deer-4"
4,Cage934,"Zebra-12, Gorilla-8, Chimpanzee-10"


In [4]:
df = data.copy()
df = pd.read_excel('PQ-297.xlsx', usecols = 'A:B', header = 0, nrows = 5)
df['Animals & Count'] = df['Animals & Count'].apply(split_values)
df = df.explode('Animals & Count')
df[['Animals', 'Count']] = pd.DataFrame(df['Animals & Count'].tolist(), index = df.index)
df[['Cage', 'CageNumber']] = df['Cage'].str.extract(r'([A-Za-z]+)(\d+)', expand = True)
df = df.astype({'Count':int, 'CageNumber':int})
df = df.sort_values(['CageNumber', 'Animals'])
df['N'] = df.groupby('CageNumber')['CageNumber'].cumcount().add(1)
df['Cage'] = df['Cage'] + df['CageNumber'].astype(str) + '_' + df['N'].astype(str)
df = df.reset_index(drop = True)
df = df.drop(['CageNumber', 'N', 'Animals & Count'], axis = 1)
df

,Cage,Animals,Count
0,Cage2_1,Cheetah,1
1,Cage2_2,Wolf,8
2,Cage3_1,Jaguar,2
3,Cage10_1,Elephant,4
4,Cage10_2,Leopard,1
5,Cage10_3,Lion,2
6,Cage10_4,Panther,1
7,Cage10_5,Tiger,1
8,Cage12_1,Deer,4
9,Cage12_2,Giraffe,6


##### v2

In [5]:
df = data.copy()

df = (
    df
    .assign(**{'Animals & Count': df['Animals & Count'].apply(split_values)})
    .explode('Animals & Count')
)

df[['Animals', 'Count']] = pd.DataFrame(df['Animals & Count'].tolist(), index = df.index)
df[['Cage', 'CageNumber']] = df['Cage'].str.extract(r'([A-Za-z]+)(\d+)', expand = True)

df = (
    df
    .astype({'Count':int, 'CageNumber':int})
    .sort_values(['CageNumber', 'Animals'])
    # .pipe(lambda row: row.assign(N = row.groupby('CageNumber')['CageNumber'].cumcount().add(1)))
    .assign(N = lambda d: d.groupby('CageNumber')['CageNumber'].cumcount().add(1))
    .assign(Cage = lambda d: d['Cage'] + d['CageNumber'].astype(str) + '_' + d['N'].astype(str))
    .reset_index(drop = True)
    .drop(['CageNumber', 'N', 'Animals & Count'], axis = 1)
)
df

,Cage,Animals,Count
0,Cage2_1,Cheetah,1
1,Cage2_2,Wolf,8
2,Cage3_1,Jaguar,2
3,Cage10_1,Elephant,4
4,Cage10_2,Leopard,1
5,Cage10_3,Lion,2
6,Cage10_4,Panther,1
7,Cage10_5,Tiger,1
8,Cage12_1,Deer,4
9,Cage12_2,Giraffe,6
